# flower classifier with PyTorch

for every project we should follow *three* steps:
#### 1. preparing the data
#### 2. create a model
#### 3. evaluate the model

## 1. Preparing the data

for this project we have a flower dataset which contains 17 types of flower that exist in *17_flowers* directory.
the dataset contains *train* and *test* data.

for reading data and prepare those, there are different ways, for example: using glob, using os, using pytorch, ...
here we use pythorch for reading and preparing the data.

In [1]:
import torch

# set device for modeling
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

### 1.1_ using torchvision.transforms to build train and test transforms

In [2]:
from torchvision import transforms

train_dir = '17_flowers/train'
test_dir = '17_flowers/test'

In [3]:
train_transform = transforms.Compose([transforms.Resize(size=(64, 64)), 
                                     transforms.RandomHorizontalFlip(0.5), 
                                     transforms.ToTensor()])

test_transform = transforms.Compose([transforms.Resize(size=(64, 64)), 
                                    transforms.ToTensor()])

### 1.2_ using torchvision.datasets.ImageFolder() to create torch dataset from our data and then split them to batches with torch.utils.data.DataLoader().

In [4]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

def create_dataset(train_dir, test_dir, train_transform, test_transform, batch_size, shuffle):
    train_data = ImageFolder(root=train_dir, transform=train_transform)
    test_data = ImageFolder(root=test_dir, transform=test_transform)
    
    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=shuffle)
    test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)
    
    
    return train_loader, test_loader, train_data.classes

In [5]:
BATCH_SIZE = 64

train_loader, test_loader, classes = create_dataset(train_dir=train_dir, 
                                                   test_dir=test_dir, 
                                                   train_transform=train_transform, 
                                                   test_transform=test_transform, 
                                                   batch_size=BATCH_SIZE,
                                                   shuffle=True)
classes

FileNotFoundError: [WinError 3] The system cannot find the path specified: '17_flowers/test'

In [ ]:
# see an image in dataloader
img = next(iter(train_loader))[0][0]

img, img.shape

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img.permute(1, 2, 0))

Here we prepare our data to tensor data and everything is ok to go through the next step!

## 2. Create a Model

for creating a model there is a few ways. we build a model with creating a class.

### 2.1_ nn.Module

the class should inherit the *nn.Module* from torch for the base model.

In [ ]:
import torch.nn as nn

class Flower_classifier(nn.Module):
    def __init__(self,
                 input_shape: int, 
                 output_shape: int):
        super().__init__()
        
        self.block_1 = nn.Sequential(nn.Conv2d(in_channels=input_shape, out_channels=32, kernel_size=(3, 3), padding=1), 
                                     nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), padding=1), 
                                     nn.ReLU(), 
                                     nn.MaxPool2d((2, 2)))
        
        self.block_2 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1), 
                                     nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1), 
                                     nn.ReLU(), 
                                     nn.MaxPool2d((2, 2)))
            
        self.block_3 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1), 
                                     nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1), 
                                     nn.ReLU(), 
                                     nn.MaxPool2d((2, 2)))
        
        self.classifier = nn.Sequential(nn.Flatten(), 
                                        nn.Linear(in_features=8*8*128, out_features=output_shape))
        
    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        x = self.classifier(x)
        return x

In [ ]:
# create an instance of our model.

torch.manual_seed(42)

model = Flower_classifier(input_shape=3, 
                          output_shape = len(classes)).to(device)

model

In [ ]:
# testing our model to see it built correctly.

pred = model(img.unsqueeze(0))
pred

### 2.2_ Optimizer, Loss function, Metrics

now after buliding a model, we should create optimizer, loss function and metrics like accuracy score.

In [ ]:
from torch import optim
from torch.nn import CrossEntropyLoss

optimizer = optim.SGD(model.parameters(), lr=0.001)
loss_fn = CrossEntropyLoss()

def acc_fn(y_pred, y_true):
    correct = torch.eq(y_pred, y_true).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc